In [1]:
import cv2
import numpy as np
import imutils #Instalar librería para establecer el tamaño del video

In [2]:
cap = cv2.VideoCapture('Carros.mp4')
carros = cv2.CascadeClassifier('cars.xml')

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG() #Es necesario instalar la dependencia "contrib"
#ya que jupyter no la incluye
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
car_counter = 0

while True:
    ret, frame = cap.read()
    if ret == False: break
    frame = imutils.resize(frame, width=640)
    
    #Especifica los puntos a del area a analizar
    area_pts = np.array([[330, 10], [frame.shape[1]-80, 10], [frame.shape[1]-80, 355], [330, 355]])
    
    #Detector de movimiento
    imAux = np.zeros(shape=(frame.shape[:2]), dtype=np.uint8)
    imAux = cv2.drawContours(imAux, [area_pts], -1, (255), -1)
    image_area = cv2.bitwise_and(frame, frame, mask=imAux)
    
    #Sustracción de fondo
    fgmask = fgbg.apply(image_area)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    fgmask = cv2.dilate(fgmask, None, iterations=5)
    
    #Encontrar contornos presentes
    cnts = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    for cnt in cnts:
        if cv2.contourArea(cnt) > 1500:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,255), 1)
            
            #Incrementar contador
            if 440 < (x + w) < 460:
                car_counter = car_counter + 1
                cv2.line(frame, (450, 10), (450, 355), (0, 255, 0), 3)
    
    #Visualización
    cv2.drawContours(frame, [area_pts], -1, (0, 128, 0), 2)
    cv2.line(frame, (450, 10), (450, 355), (0, 255, 255), 1)
    cv2.rectangle(frame, (frame.shape[1]-70, 215), (frame.shape[1]-5, 270), (0, 255, 0), 2)
    cv2.putText(frame, str(car_counter), (frame.shape[1]-55, 250),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)
    cv2.imshow('Frame', frame)
    #cv2.imshow('fgmask', fgmask)
        
    k = cv2.waitKey(70) & 0xFF
    if k == 27:
        break
            
cap.release()
cv2.destroyAllWindows()